# **🏠 부동산 실거래가 Baseline code**
> 부동산 실거래가 예측 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 RandomForest를 활용해 ML 방법론들을 실제 대회에 적용해보도록 하겠습니다.
> 강의는 google colab으로 실행하였기에 아래의 베이스라인 코드와는 일부 차이가 있을 수 있습니다. 

## Contents
- Library Import
- Data Load
- Data Preprocessing
- Feature Engineering
- Model Training
- Inference
- Output File Save


## 1. Library Import
- 필요한 라이브러리를 불러옵니다.

In [ ]:
!pip install eli5==0.13.0

# 한글 폰트 사용을 위한 라이브러리입니다.
!apt-get install -y fonts-nanum

In [18]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

pd.set_option('display.float_format', '{:.6f}'.format)

## 2. Data Load

#### 2.1. 데이터 로드

In [ ]:
import os
os.getcwd()

### dataset_2 시작

In [189]:
concat = pd.read_csv('../data/base_dataset_3.csv')

concat[concat['is_test'] == 1].shape

(9272, 26)

In [190]:
concat.shape

(1128094, 26)

In [158]:
concat[concat['is_test'] == 1].head()

,번지,본번,부번,전용면적,계약일,층,건축년도,target,is_test,주소,...,구,동,강남여부,신축여부,건축면적,연면적,대지면적,건폐율,용적율,평균층수
1118822,658-1,658,1,79.970000,26,5,1987,NaN,1,서울특별시 강남구 개포동 658-1,...,강남구,개포동,1,0,4551.330000,22305.930000,171357.334166,297.555783,3143.834230,4.900970
1118823,651-1,651,1,108.201700,15,10,2021,NaN,1,서울특별시 강남구 개포동 651-1,...,강남구,개포동,1,1,2706.031700,31893.018900,164240.992757,297.402886,3140.519263,11.785900
1118824,652,652,0,161.000000,28,15,1984,NaN,1,서울특별시 강남구 개포동 652,...,강남구,개포동,1,0,4484.940000,61999.080000,183006.694768,299.033944,3163.697455,13.823837
1118825,652,652,0,133.460000,10,14,1984,NaN,1,서울특별시 강남구 개포동 652,...,강남구,개포동,1,0,4484.940000,61999.080000,183006.694768,299.033944,3163.697455,13.823837
1118826,652,652,0,104.430000,18,6,1984,NaN,1,서울특별시 강남구 개포동 652,...,강남구,개포동,1,0,4484.940000,61999.080000,183006.694768,299.033944,3163.697455,13.823837


In [23]:
# gis = pd.read_csv('../data/gis건물정보_fixed.csv')

In [24]:
# change_col = ['건축면적', '연면적', '대지면적', '건폐율', '용적율', '평균층수']

# # concat.drop(columns='평균층수', inplace=True)

# congis = concat.drop(columns=change_col)
# congis.set_index('주소', inplace=True)

# gis.set_index('ID', inplace=True)

# for col in change_col:
#     congis[col] = congis.index.map(gis[col])

# congis.reset_index(inplace=True)

# concat = congis

In [ ]:
# congis.columns

In [ ]:
# con_temp = concat[concat['is_test'] == 1]
# con_temp.head(10)

In [25]:
# # 주소정렬 후 결측치 채움

# congis = concat.sort_values('주소')

# col = ['건축면적', '연면적', '대지면적', '건폐율', '용적율', '평균층수']
# congis[col] = congis[col].interpolate(method='linear', axis=0)

# congis.sort_index(inplace=True)

# # con_temp['건폐율'].describe()
# concat = congis

In [ ]:
# congis['연면적'].describe()

In [ ]:
# con_temp = concat[concat['is_test'] == 1]
# con_temp.head(10)

In [191]:
concat['계약월'] = concat['계약월'].apply(lambda x: f'{x:02d}')

In [192]:
concat['계약월'].head()

0    12
1    12
2    12
3    01
4    01
Name: 계약월, dtype: object

In [194]:
concat['계약년월'] = concat['계약년'].astype(str) + '' + concat['계약월'].astype(str)

interest = pd.read_csv('../data/한국기준금리07-23.csv')

interest['날짜'] = interest['날짜'].astype(str)

interest = interest.drop_duplicates(subset=['날짜'], keep='first')

interest['계약년월'] = interest['날짜'].str[:6]
interest.drop(columns = '날짜', inplace = True)

In [196]:
concat['계약년월'] = concat['계약년월'].astype(str)

concat_temp = pd.merge(concat, interest, on='계약년월', how='left', suffixes=('_A', '_B'))

In [195]:
concat_temp

0    201712
1    201712
2    201712
3    201801
4    201801
Name: 계약년월, dtype: object

In [186]:
# 금리데이터를 넣습니다



concat['계약년월'] = concat['계약년'].astype(str) + '' + concat['계약월'].astype(str)

interest = pd.read_csv('../data/한국기준금리07-23.csv')

interest['날짜'] = interest['날짜'].astype(str)

interest = interest.drop_duplicates(subset=['날짜'], keep='first')

interest['계약년월'] = interest['날짜'].str[:6]
interest.drop(columns = '날짜', inplace = True)
# interest['계약년월'].fillna('-1')
# interest['계약년월'] = interest['계약년월'].astype(str)
concat['계약년월'] = concat['계약년월'].astype(str)

concat_temp = pd.merge(concat, interest, on='계약년월', how='left', suffixes=('_A', '_B'))

concat_temp.drop(columns='계약년월', inplace=True)
# concat = concat_temp

In [160]:
concat[concat['is_test'] == 1].shape

(9272, 28)

In [188]:
concat_temp.shape

(1174885, 27)

In [162]:
concat

Index(['번지', '본번', '부번', '전용면적', '계약일', '층', '건축년도', 'target', 'is_test', '주소',
       'x', 'y', '계약년', '계약월', '거래취소여부', '거래일건물연식', '구', '동', '강남여부', '신축여부',
       '건축면적', '연면적', '대지면적', '건폐율', '용적율', '평균층수', '계약년월', '기준금리'],
      dtype='object')

In [161]:
concat[concat['is_test'] == 1].head()

,번지,본번,부번,전용면적,계약일,층,건축년도,target,is_test,주소,...,강남여부,신축여부,건축면적,연면적,대지면적,건폐율,용적율,평균층수,계약년월,기준금리
1129190,658-1,658,1,79.970000,26,5,1987,NaN,1,서울특별시 강남구 개포동 658-1,...,1,0,4551.330000,22305.930000,171357.334166,297.555783,3143.834230,4.900970,20237,NaN
1129191,651-1,651,1,108.201700,15,10,2021,NaN,1,서울특별시 강남구 개포동 651-1,...,1,1,2706.031700,31893.018900,164240.992757,297.402886,3140.519263,11.785900,20238,NaN
1129192,652,652,0,161.000000,28,15,1984,NaN,1,서울특별시 강남구 개포동 652,...,1,0,4484.940000,61999.080000,183006.694768,299.033944,3163.697455,13.823837,20237,NaN
1129193,652,652,0,133.460000,10,14,1984,NaN,1,서울특별시 강남구 개포동 652,...,1,0,4484.940000,61999.080000,183006.694768,299.033944,3163.697455,13.823837,20238,NaN
1129194,652,652,0,104.430000,18,6,1984,NaN,1,서울특별시 강남구 개포동 652,...,1,0,4484.940000,61999.080000,183006.694768,299.033944,3163.697455,13.823837,20238,NaN


In [ ]:
# concat_temp['연면적'].describe()

In [143]:
# 평형별로 초소형, 소형, 중형, 대형으로 분류

bins = [0, 26, 50, 60, 85, 135, float('inf')]
labels = [0, 1, 2, 3, 4, 5]

concat['면적분류'] = pd.cut(concat['전용면적'], bins=bins, labels=labels, right=False)

In [63]:
# concat['original_index'] = concat.index

concat['계약날짜'] = concat['계약년'].astype(str) + '-' + concat['계약월'].astype(str) + '-' + concat['계약일'].astype(str)

concat['계약날짜'] = pd.to_datetime(concat['계약날짜'])
concat.set_index('계약날짜', inplace=True)

daily_trades = concat.groupby(concat.index).size()

# 이동평균 만들기

concat['3m이동평균'] = concat['target'].resample('D').mean().rolling(window=90).mean()
concat['12m이동평균'] = concat['target'].resample('D').mean().rolling(window=360).mean()
concat['48m이동평균'] = concat['target'].resample('D').mean().rolling(window=1440).mean()

# 일별거래량 데이터 만들기

day_t = pd.DataFrame(daily_trades, columns=['trades'])

day_t['30d_trades'] = day_t['trades'].rolling(window=30).sum()

concat['30d_trades'] = concat.index.map(day_t['30d_trades'])
concat['day_trades'] = concat.index.map(day_t['trades'])

# concat.head()
concat.reset_index(inplace=True)
concat.drop(columns='계약날짜', inplace=True)

In [ ]:
# con_temp = concat[concat['is_test'] == 1]
# con_temp.head(10)

In [74]:
concat.head()

,번지,본번,부번,전용면적,계약일,층,건축년도,target,is_test,주소,...,연면적,대지면적,건폐율,용적율,평균층수,계약년월,기준금리,면적분류,30d_trades,day_trades
0,658-1,658,1,79.970000,8,3,1987,124000.000000,0,서울특별시 강남구 개포동 658-1,...,22305.930000,171357.334166,297.555783,3143.834230,4.900970,201712,1.500000,3,8123.000000,287
1,658-1,658,1,79.970000,22,4,1987,123500.000000,0,서울특별시 강남구 개포동 658-1,...,22305.930000,171357.334166,297.555783,3143.834230,4.900970,201712,1.500000,3,8100.000000,291
2,658-1,658,1,54.980000,28,5,1987,91500.000000,0,서울특별시 강남구 개포동 658-1,...,22305.930000,171357.334166,297.555783,3143.834230,4.900970,201712,1.500000,2,8225.000000,294
3,658-1,658,1,79.970000,3,4,1987,130000.000000,0,서울특별시 강남구 개포동 658-1,...,22305.930000,171357.334166,297.555783,3143.834230,4.900970,20181,NaN,3,8016.000000,298
4,658-1,658,1,79.970000,8,2,1987,117000.000000,0,서울특별시 강남구 개포동 658-1,...,22305.930000,171357.334166,297.555783,3143.834230,4.900970,20181,NaN,3,8423.000000,437


In [ ]:
# #1일 평균가격

# daily_price = concat.groupby(concat.index)['target'].mean()

# concat['day_price'] = concat.index.map(daily_price)

# #월 평균가격
# monthly_price = concat.groupby('계약년월')['target'].mean()

# concat['month_price'] = concat['계약년월'].map(monthly_price)

# concat.head()

In [ ]:
# #작년 동월 기준 변화량

# monthly_avg = concat.groupby('계약년월')['target'].mean()

# monthly_per = (monthly_avg - monthly_avg.shift(12)) / monthly_avg.shift(12) * 100

# concat['작년동월변화량'] = concat['계약년월'].map(monthly_per)

# #지난달 기준 변화량

# monthly_avg = concat.groupby('계약년월')['target'].mean()

# month_diff = (monthly_avg - monthly_avg.shift(1)) / monthly_avg.shift(1) * 100

# concat['지난달변화량'] = concat['계약년월'].map(month_diff)

In [144]:
str_columns = ['주소', '본번', '부번', '거래취소여부', '구', '동', '강남여부', '신축여부']

concat[str_columns] = concat[str_columns].astype(str)

In [145]:
# 먼저, 연속형 변수와 범주형 변수를 위 info에 따라 분리해주겠습니다.
continuous_columns = []
categorical_columns = []

for column in concat.columns:
    if pd.api.types.is_numeric_dtype(concat[column]):
        continuous_columns.append(column)
    else:
        categorical_columns.append(column)

print("연속형 변수:", continuous_columns)
print("범주형 변수:", categorical_columns)



연속형 변수: ['전용면적', '계약일', '층', '건축년도', 'target', 'is_test', 'x', 'y', '계약년', '계약월', '거래일건물연식', '건축면적', '연면적', '대지면적', '건폐율', '용적율', '평균층수']
범주형 변수: ['번지', '본번', '부번', '주소', '거래취소여부', '구', '동', '강남여부', '신축여부', '면적분류']


- 연속형 변수와 범주형 변수가 알맞게 나누어졌는지 확인해보고, 이제 각각 보간을 진행합니다.

In [7]:
# # 범주형 변수에 대한 보간
# concat[categorical_columns] = concat[categorical_columns].fillna('NULL')

# # 연속형 변수에 대한 보간 (선형 보간)
# concat[continuous_columns] = concat[continuous_columns].interpolate(method='linear', axis=0)

In [ ]:
concat.isnull().sum()         # 결측치가 보간된 모습을 확인해봅니다.

In [10]:
concat.columns

Index(['번지', '본번', '부번', '전용면적', '계약일', '층', '건축년도', 'target', 'is_test', '주소',
       'x', 'y', '계약년', '계약월', '거래취소여부', '거래일건물연식', '구', '동', '강남여부', '신축여부',
       '건축면적', '연면적', '대지면적', '건폐율', '용적율', '평균층수', '계약년월', '기준금리', '면적분류'],
      dtype='object')

In [ ]:
# 이상치 제거 이전의 shape은 아래와 같습니다.
print(concat.shape)

연속형 변수: ['전용면적', '계약일', '층', '건축년도', 'target', 'x', 'y', '계약년', '계약월', '거래일건물연식', '건축면적', '연면적', '대지면적', '건폐율', '용적율', '평균층수', '지하철최단거리', '버스최단거리']
범주형 변수: ['번지', '본번', '부번', '주소', '거래취소여부', '구', '동', '강남여부', '신축여부']

In [ ]:
# 대표적인 연속형 변수인 “전용 면적” 변수 관련한 분포를 먼저 살펴보도록 하겠습니다.
fig = plt.figure(figsize=(7, 3))
sns.boxplot(data = concat, x = '건폐율', color='lightgreen')
plt.title('건폐율')
plt.xlabel('Area')
plt.show()

print(concat['연면적'].describe())

In [ ]:
# 대표적인 연속형 변수인 “전용 면적” 변수 관련한 분포를 먼저 살펴보도록 하겠습니다.
fig = plt.figure(figsize=(7, 3))
sns.boxplot(data = concat, x = '건축년도', color='lightgreen')
plt.title('건축년도 분포')
plt.xlabel('Area')
plt.show()

In [ ]:
# 대표적인 연속형 변수인 “전용 면적” 변수 관련한 분포를 먼저 살펴보도록 하겠습니다.
fig = plt.figure(figsize=(7, 3))
sns.boxplot(data = concat, x = '계약년', color='lightgreen')
plt.title('전용면적 분포')
plt.xlabel('Area')
plt.show()

In [ ]:
# 대표적인 연속형 변수인 “전용 면적” 변수 관련한 분포를 먼저 살펴보도록 하겠습니다.
fig = plt.figure(figsize=(7, 3))
sns.boxplot(data = concat, x = '전용면적', color='lightgreen')
plt.title('전용면적 분포')
plt.xlabel('Area')
plt.show()

In [ ]:
# concat.columns

In [ ]:
# concat['층'].describe()

In [146]:
# 이상치 제거 방법에는 IQR을 이용하겠습니다.
def remove_outliers_iqr(dt, column_name):
    df = dt.query('is_test == 0')    # train data 내에 있는 이상치만 제거하도록 하겠습니다.
    df_test = dt.query('is_test == 1')

    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

    result = pd.concat([df, df_test])   # test data와 다시 합쳐주겠습니다.
    return result

#### 결측치 처리

In [147]:
# test와 train 분리
df = concat.query('is_test == 0')  
df_test = concat.query('is_test == 1')  

# 전용면적 375 이상인 것들은 제거
df= df[df['전용면적'] < 375]

# 층 음수 처리
df['층'] = df['층'].apply(lambda x: 1 if x < 0 else x)
df_test['층'] = df_test['층'].apply(lambda x: 1 if x < 0 else x)

# 건축년도 1975년도 이후에 지어진것만 처리
df= df[df['건축년도'] > 1975]

# 거래일 건물 연식 음수 처리
df= df[df['거래일건물연식'] > 0]
concat_select = pd.concat([df, df_test])

In [16]:
# 위 방법으로 전용 면적에 대한 이상치를 제거해보겠습니다.
#concat_select = remove_outliers_iqr(concat, '전용면적')

In [ ]:
concat_select['is_test'].value_counts()     # 또한, train data만 제거되었습니다.

In [ ]:
concat_select.info()       # 최종 데이터셋은 아래와 같습니다.

- 이제 위에서 만든 파생변수들과 정제한 데이터를 기반으로 본격적으로 부동산 실거래가를 예측하는 모델링을 진행하겠습니다.
- 모델링에는 `sklearn`의 `RandomForest`를 이용하도록 하겠습니다.

- 참고 ✅
  - `RandomForest`는 배깅(Bagging)의 일종으로, 학습시키는 데이터 뿐 아니라 특성변수(X)들도 무작위로 선택해 트리를 생성하는 방법입니다.
  - 모델 학습 과정에서 서로 다른 N개의 Tree 생성하고, N개의 Tree에서 발생한 Output을 Voting(범주형, 분류문제)하거나, Average(연속형, 회귀문제)해 최종 Output 생성합니다.
  - 이는 High variance, Low bias 상황에서 분산(Variance) 감소에 도움을 줍니다.

### 대중교통 파생변수 생성



In [34]:
from math import radians, sin, cos, sqrt, atan2

# 하버사인 공식으로 두 지리적 좌표 간의 거리를 계산하는 함수
def haversine_np(lon1, lat1, lon2, lat2):
    R = 6371  # 지구 반경 (킬로미터)
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance
subway_path = pd.read_csv('../data/subway_feature.csv')
bus_path = pd.read_csv('../data/bus_feature.csv')
# concat_select 좌표와 subway_path 좌표를 numpy 배열로 변환
concat_coords = np.array([concat_select['x'], concat_select['y']]).T
subway_coords = np.array([subway_path['경도'], subway_path['위도']]).T
bus_coords = np.array([bus_path['X좌표'], bus_path['Y좌표']]).T
# 빈 리스트를 만들어서 최단 거리를 저장할 것임
shortest_distances_subway = []
shortest_distances_bus = []
# 벡터 연산을 사용하여 가장 가까운 지하철 거리 찾기
for i, concat_coord in tqdm(enumerate(concat_coords), total=len(concat_coords)):
    distances = haversine_np(concat_coord[0], concat_coord[1], subway_coords[:, 0], subway_coords[:, 1])
    min_distance = np.min(distances)
    shortest_distances_subway.append(min_distance)
# 벡터 연산을 사용하여 가장 가까운 버스 거리 찾기
for i, concat_coord in tqdm(enumerate(concat_coords), total=len(concat_coords)):
    distances = haversine_np(concat_coord[0], concat_coord[1], bus_coords[:, 0], bus_coords[:, 1])
    min_distance = np.min(distances)
    shortest_distances_bus.append(min_distance)
# concat_select에 최단 거리를 추가
concat_select['지하철최단거리'] = shortest_distances_subway
concat_select['버스최단거리'] = shortest_distances_bus
concat_select['버스최단거리'].describe()

#거리별 가중치 부여
concat_select['지하철최단거리'] = concat_select['지하철최단거리'].apply(lambda x: 3 if 0 < x < 0.3 else (2 if 0.3 < x < 0.65 else(1 if 0.65<x<1.0 else 0) ))
concat_select['버스최단거리'] = concat_select['버스최단거리'].apply(lambda x: 2 if 0 < x < 0.5 else (1 if 0.5 < x < 1 else 0))


#10분 이내로 

100%|██████████| 1096992/1096992 [11:01<00:00, 1657.54it/s]


In [148]:
# cong = concat_select
concat_select['지하철최단거리'] = cong['지하철최단거리']
concat_select['버스최단거리'] = cong['버스최단거리']
concat_select.head()

,번지,본번,부번,전용면적,계약일,층,건축년도,target,is_test,주소,...,신축여부,건축면적,연면적,대지면적,건폐율,용적율,평균층수,면적분류,지하철최단거리,버스최단거리
0,658-1,658,1,79.970000,8,3,1987,124000.000000,0,서울특별시 강남구 개포동 658-1,...,0,4551.330000,22305.930000,171357.334166,297.555783,3143.834230,4.900970,3,0,2
1,658-1,658,1,79.970000,22,4,1987,123500.000000,0,서울특별시 강남구 개포동 658-1,...,0,4551.330000,22305.930000,171357.334166,297.555783,3143.834230,4.900970,3,0,2
2,658-1,658,1,54.980000,28,5,1987,91500.000000,0,서울특별시 강남구 개포동 658-1,...,0,4551.330000,22305.930000,171357.334166,297.555783,3143.834230,4.900970,2,0,2
3,658-1,658,1,79.970000,3,4,1987,130000.000000,0,서울특별시 강남구 개포동 658-1,...,0,4551.330000,22305.930000,171357.334166,297.555783,3143.834230,4.900970,3,0,2
4,658-1,658,1,79.970000,8,2,1987,117000.000000,0,서울특별시 강남구 개포동 658-1,...,0,4551.330000,22305.930000,171357.334166,297.555783,3143.834230,4.900970,3,0,2


In [149]:
# 학교와 거리 구하기
school_path = pd.read_csv('./data/청주대학교_지방교육재정연구원_초중등학교위치_20240322.csv', encoding='EUC-KR')

h_school = school_path[school_path['학교급구분'] == '고등학교']
m_school = school_path[school_path['학교급구분'] == '중학교']
p_school = school_path[school_path['학교급구분'] == '초등학교']

h_school_coords = np.array([h_school['경도'], h_school['위도']]).T
m_school_coords = np.array([m_school['경도'], m_school['위도']]).T
p_school_coords = np.array([p_school['경도'], p_school['위도']]).T

# 빈 리스트를 만들어서 최단 거리를 저장할 것임
shortest_distances_h_school = []
shortest_distances_m_school = []
shortest_distances_p_school = []

for i, concat_coord in tqdm(enumerate(concat_coords), total=len(concat_coords)):
    distances = haversine_np(concat_coord[0], concat_coord[1], h_school_coords[:, 0], h_school_coords[:, 1])
    min_distance1 = np.min(distances)
    shortest_distances_h_school.append(min_distance1)

for i, concat_coord in tqdm(enumerate(concat_coords), total=len(concat_coords)):
    distances = haversine_np(concat_coord[0], concat_coord[1], m_school_coords[:, 0], m_school_coords[:, 1])
    min_distance2 = np.min(distances)
    shortest_distances_m_school.append(min_distance2)

for i, concat_coord in tqdm(enumerate(concat_coords), total=len(concat_coords)):
    distances = haversine_np(concat_coord[0], concat_coord[1], p_school_coords[:, 0], p_school_coords[:, 1])
    min_distance3 = np.min(distances)
    shortest_distances_p_school.append(min_distance3)

concat_select['고등학교최단거리'] = shortest_distances_h_school
concat_select['중등학교최단거리'] = shortest_distances_m_school
concat_select['초등학교최단거리'] = shortest_distances_p_school

concat_select['초등학교최단거리'] = concat_select['초등학교최단거리'].apply(lambda x: 3 if 0 < x < 0.5 else (2 if 0.5 < x < 1.0 else(1 if 1.0<x<1.5 else 0) ))
concat_select['중등학교최단거리'] = concat_select['중등학교최단거리'].apply(lambda x: 3 if 0 < x < 0.5 else (2 if 0.5 < x < 1.0 else(1 if 1.0<x<1.5 else 0) ))
concat_select['고등학교최단거리'] = concat_select['고등학교최단거리'].apply(lambda x: 3 if 0 < x < 0.5 else (2 if 0.5 < x < 1.0 else(1 if 1.0<x<1.5 else 0) ))

concat_select.head()

100%|██████████| 1096992/1096992 [05:50<00:00, 3126.76it/s]


,번지,본번,부번,전용면적,계약일,층,건축년도,target,is_test,주소,...,대지면적,건폐율,용적율,평균층수,면적분류,지하철최단거리,버스최단거리,고등학교최단거리,중등학교최단거리,초등학교최단거리
0,658-1,658,1,79.970000,8,3,1987,124000.000000,0,서울특별시 강남구 개포동 658-1,...,171357.334166,297.555783,3143.834230,4.900970,3,0,2,3,1,3
1,658-1,658,1,79.970000,22,4,1987,123500.000000,0,서울특별시 강남구 개포동 658-1,...,171357.334166,297.555783,3143.834230,4.900970,3,0,2,3,1,3
2,658-1,658,1,54.980000,28,5,1987,91500.000000,0,서울특별시 강남구 개포동 658-1,...,171357.334166,297.555783,3143.834230,4.900970,2,0,2,3,1,3
3,658-1,658,1,79.970000,3,4,1987,130000.000000,0,서울특별시 강남구 개포동 658-1,...,171357.334166,297.555783,3143.834230,4.900970,3,0,2,3,1,3
4,658-1,658,1,79.970000,8,2,1987,117000.000000,0,서울특별시 강남구 개포동 658-1,...,171357.334166,297.555783,3143.834230,4.900970,3,0,2,3,1,3


In [150]:
aa11 = concat_select

In [ ]:
[aa11['초등학교최단거리']]

In [129]:
concat_select['초등학교최단거리'].describe()

count   1096992.000000
mean          2.854883
std           0.357368
min           0.000000
25%           3.000000
50%           3.000000
75%           3.000000
max           3.000000
Name: 초등학교최단거리, dtype: float64

In [130]:
# cons = concat_select

# concat_select['고등학교최단거리'] = cons['고등학교최단거리']
# concat_select['중등학교최단거리'] = cons['중등학교최단거리']
# concat_select['초등학교최단거리'] = cons['초등학교최단거리']



concat_select['초등학교최단거리'] = concat_select['초등학교최단거리'].apply(lambda x: 3 if 0 < x < 0.5 else (2 if 0.5 < x < 1.0 else 0))
concat_select['중등학교최단거리'] = concat_select['중등학교최단거리'].apply(lambda x: 3 if 0 < x < 0.5 else (2 if 0.5 < x < 1.0 else(1 if 1.0<x<1.5 else 0) ))
concat_select['고등학교최단거리'] = concat_select['고등학교최단거리'].apply(lambda x: 3 if 0 < x < 0.5 else (2 if 0.5 < x < 1.0 else(1 if 1.0<x<1.5 else 0) ))

In [131]:
concat_select.isnull().sum()

번지             0
본번             0
부번             0
전용면적           0
계약일            0
층              0
건축년도           0
target      9272
is_test        0
주소             0
x              0
y              0
계약년            0
계약월            0
거래취소여부         0
거래일건물연식        0
구              0
동              0
강남여부           0
신축여부           0
건축면적           0
연면적            0
대지면적           0
건폐율            0
용적율            0
평균층수           0
지하철최단거리        0
버스최단거리         0
고등학교최단거리       0
중등학교최단거리       0
초등학교최단거리       0
dtype: int64

In [175]:
str_col = ['지하철최단거리', '버스최단거리', '고등학교최단거리', '중등학교최단거리', '초등학교최단거리']

concat_select[str_col] = concat_select[str_col].astype(str)

In [176]:
# 이제 다시 train과 test dataset을 분할해줍니다. 위에서 제작해 놓았던 is_test 칼럼을 이용합니다.
dt_train = concat_select.query('is_test==0')
dt_test = concat_select.query('is_test==1')

# 이제 is_test 칼럼은 drop해줍니다.
dt_train.drop(['is_test'], axis = 1, inplace=True)
dt_test.drop(['is_test'], axis = 1, inplace=True)
print(dt_train.shape, dt_test.shape)

# dt_test의 target은 일단 0으로 임의로 채워주도록 하겠습니다.
dt_test['target'] = 0

(1087720, 31) (9272, 31)


### 5.1. 범주형 변수 Encoding
- 범주형 변수는 그대로 모델에 투입하면, 모델이 제대로 작동할 수 없습니다.
- 따라서 **레이블 인코딩 과정**을 통해 범주형 변수들을 numeric하게 바꾸는 인코딩 과정을 진행해주도록 하겠습니다.

In [152]:
# 파생변수 제작으로 추가된 변수들이 존재하기에, 다시한번 연속형과 범주형 칼럼을 분리해주겠습니다.
continuous_columns_v2 = []
categorical_columns_v2 = []

for column in dt_train.columns:
    if pd.api.types.is_numeric_dtype(dt_train[column]):
        continuous_columns_v2.append(column)
    else:
        categorical_columns_v2.append(column)

print("연속형 변수:", continuous_columns_v2)
print("범주형 변수:", categorical_columns_v2)

# 아래에서 범주형 변수들을 대상으로 레이블인코딩을 진행해 주겠습니다.

# 각 변수에 대한 LabelEncoder를 저장할 딕셔너리
label_encoders = {}

# Implement Label Encoding
for col in tqdm( categorical_columns ):
    lbl = LabelEncoder()

    # Label-Encoding을 fit
    lbl.fit( dt_train[col].astype(str) )
    dt_train[col] = lbl.transform(dt_train[col].astype(str))
    label_encoders[col] = lbl           # 나중에 후처리를 위해 레이블인코더를 저장해주겠습니다.

    # Test 데이터에만 존재하는 새로 출현한 데이터를 신규 클래스로 추가해줍니다.
    for label in np.unique(dt_test[col]):
      if label not in lbl.classes_: # unseen label 데이터인 경우
        lbl.classes_ = np.append(lbl.classes_, label) # 미처리 시 ValueError발생하니 주의하세요!

    dt_test[col] = lbl.transform(dt_test[col].astype(str))

연속형 변수: ['전용면적', '계약일', '층', '건축년도', 'target', 'x', 'y', '계약년', '계약월', '거래일건물연식', '건축면적', '연면적', '대지면적', '건폐율', '용적율', '평균층수', '지하철최단거리', '버스최단거리', '고등학교최단거리', '중등학교최단거리', '초등학교최단거리']
범주형 변수: ['번지', '본번', '부번', '주소', '거래취소여부', '구', '동', '강남여부', '신축여부', '면적분류']


100%|██████████| 10/10 [00:02<00:00,  3.44it/s]


In [153]:
dt_train = pd.get_dummies(dt_train, columns=['구'])
dt_test = pd.get_dummies(dt_test, columns=['구'])

#### 필요없는 데이터 삭제

In [154]:
columns_to_delete = ['번지', '부번', '본번']

# # 열 삭제
dt_train = dt_train.drop(columns=columns_to_delete)
dt_test = dt_test.drop(columns=columns_to_delete)

### 금리데이터 넣기

## 데이터 나눠서 머신러닝 돌리기

In [155]:
assert dt_train.shape[1] == dt_test.shape[1]          # train/test dataset의 shape이 같은지 확인해주겠습니다.

# Target과 독립변수들을 분리해줍니다.
y_train = dt_train['target']
X_train = dt_train.drop(['target'], axis=1)

# Hold out split을 사용해 학습 데이터와 검증 데이터를 8:2 비율로 나누겠습니다.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2023)

In [25]:
# RandomForestRegressor를 이용해 회귀 모델을 적합시키겠습니다.
#model = RandomForestRegressor(n_estimators=5, criterion='squared_error', random_state=1, n_jobs=-1)
#model.fit(X_train, y_train)
#pred = model.predict(X_val)

In [26]:
#y_val = y_val * X_val['전용면적']
#pred = pred * X_val['전용면적']

In [27]:
# 회귀 관련 metric을 통해 train/valid의 모델 적합 결과를 관찰합니다.
#print(f'RMSE test: {np.sqrt(metrics.mean_squared_error(y_val, pred))}')

In [42]:
#  XGBoost 사용

import xgboost as xgb

In [156]:
model = xgb.XGBRegressor(objective='reg:squarederror', 
                         n_estimators=2000,  
                         learning_rate=0.1,  
                         max_depth=10,
                         random_state=1)


model.fit(X_train, y_train)

pred_2 = model.predict(X_val)

# 회귀 관련 metric을 통해 train/valid의 모델 적합 결과를 관찰합니다.
print(f'RMSE test: {np.sqrt(metrics.mean_squared_error(y_val, pred_2))}')

RMSE test: 5911.546710411913


- 변수 중요도도 확인해보도록 하겠습니다.

In [ ]:
xgb.plot_importance(model)
plt.figure(figsize=(10, 100))
plt.show()

In [ ]:
xgb.plot_importance(model, importance_type='total_gain')
plt.show()

In [ ]:
# 위 feature importance를 시각화해봅니다.
importances = pd.Series(model.feature_importances_, index=list(X_train.columns))
importances = importances.sort_values(ascending=False)

plt.figure(figsize=(10,8))
plt.title("Feature Importances")
sns.barplot(x=importances, y=importances.index)
plt.show()

In [ ]:
# Permutation importance 방법을 변수 선택에 이용해보겠습니다.
perm = PermutationImportance(model,        # 위에서 학습된 모델을 이용하겠습니다.
                             scoring = "neg_mean_squared_error",        # 평가 지표로는 회귀문제이기에 negative rmse를 사용합니다. (neg_mean_squared_error : 음의 평균 제곱 오차)
                             random_state = 42,
                             n_iter=3).fit(X_val, y_val)
eli5.show_weights(perm, feature_names = X_val.columns.tolist())    # valid data에 대해 적합시킵니다.

In [ ]:
eli5.show_weights(perm, feature_names=X_val.columns.tolist(), top=X_val.shape[1])

In [42]:
# 학습된 모델을 저장합니다. Pickle 라이브러리를 이용하겠습니다.
with open('saved_model.pkl', 'wb') as f:
    pickle.dump(model, f)

### 5.4. Valid prediction 분석

- 예측값을 분석해보기 위해 valid prediction을 확인해보겠습니다.

- 예측을 잘 하지 못한 top 100개의 데이터와 예측을 잘한 top 100개의 데이터를 비교해보겠습니다.

- 이제 분포를 비교해보도록 하겠습니다.

In [34]:

# Validation dataset에 target과 pred 값을 채워주도록 하겠습니다.
X_val['target'] = y_val
X_val['pred'] = pred_2

# Squared_error를 계산하는 함수를 정의하겠습니다.
def calculate_se(target, pred):
    squared_errors = (target - pred) ** 2
    return squared_errors

# RMSE 계산
squared_errors = calculate_se(X_val['target'], X_val['pred'])
X_val['error'] = squared_errors

X_val_sort_top100 = X_val.sort_values(by='error', ascending=False).head(100)        # 예측을 잘 하지못한 top 100개의 data
X_val_sort_tail100 = X_val.sort_values(by='error', ascending=False).tail(100)       # 예측을 잘한 top 100개의 data

In [ ]:
# 해석을 위해 레이블인코딩 된 변수를 복원해줍니다.
error_top100 = X_val_sort_top100.copy()
for column in categorical_columns_v2 :     # 앞서 레이블 인코딩에서 정의했던 categorical_columns_v2 범주형 변수 리스트를 사용합니다.
    error_top100[column] = label_encoders[column].inverse_transform(X_val_sort_top100[column])

best_top100 = X_val_sort_tail100.copy()
for column in categorical_columns_v2 :     # 앞서 레이블 인코딩에서 정의했던 categorical_columns_v2 범주형 변수 리스트를 사용합니다.
    best_top100[column] = label_encoders[column].inverse_transform(X_val_sort_tail100[column])

In [37]:
best_top100 = X_val_sort_tail100.copy()
error_top100 = X_val_sort_top100.copy()

In [ ]:
sns.boxplot(data = X_val_sort_top100, x='target')
plt.title('The worst top100 prediction의 target 분포')
plt.show()

sns.boxplot(data = X_val_sort_tail100, x='target', color='orange')
plt.title('The best top100 prediction의 target 분포')
plt.show()

- Taget 분포를 보니 좋은 예측을 보인 top 100개의 data보다 상대적으로 나쁜 예측을 보인 top 100 dataset들이 높은 가격을 보였음을 확인할 수 있습니다. 이에 대한 모델링 및 처리가 필요해보입니다.

In [ ]:
sns.histplot(data = error_top100, x='전용면적', alpha=0.5)
sns.histplot(data = best_top100, x='전용면적', color='orange', alpha=0.5)
plt.title('전용면적 분포 비교')
plt.show()

- 전용면적 또한 나쁜 예측을 보인 집들이 더 넓음을 확인할 수 있습니다.

## 6. Inference

In [ ]:
dt_test.head(2)      # test dataset에 대한 inference를 진행해보겠습니다.

In [ ]:
# 저장된 모델을 불러옵니다.
with open('saved_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
%%time
X_test = dt_test.drop(['target'], axis=1)

# Test dataset에 대한 inference를 진행합니다.

real_test_pred = model.predict(X_test)

#real_test_pred = real_test_pred * dt_test['전용면적'].values

In [ ]:
real_test_pred          # 예측값들이 출력됨을 확인할 수 있습니다.

## 7. Output File Save

In [19]:
# 앞서 예측한 예측값들을 저장합니다.
preds_df = pd.DataFrame(real_test_pred.astype(int), columns=["target"])
preds_df.to_csv('output_nokumli.csv', index=False)